### Basic Try implementation of Le-Net, doesn't work that well though, (done in colab, accuracy is only 10% after ReLU ..)

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader

In [2]:
batch_size = 2 ** 8
num_classes = 10 # i.e 0-9 (output)
learning_rate = 0.001
num_epochs = 10

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')

In [4]:
device

device(type='mps')

In [5]:
transform_data = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(mean = (0.1307,), std = (0.3081,))
])

### idk why download not working in local? can someone explain...

In [6]:
train_dataset = datasets.MNIST(root="./assets", train=True, transform=transform_data, download=True)
test_dataset = datasets.MNIST(root="./assets", train=False, transform=transform_data, download=True)

Failed to download (trying next):
HTTP Error 404: Not Found

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1007)>



RuntimeError: Error downloading train-images-idx3-ubyte.gz

In [ ]:
train_loader = DataLoader(dataset = train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# defining LeNet5 arch now..
conv_kernel_size = (5, 5)
pool_kernel_size = (2, 2)

class LeNet5(nn.Module):

  def __init__(self, num_classes):
    super().__init__()

    self.c1 = nn.Conv2d(1, 6, conv_kernel_size) # with padding (0) and stride (1) default
    self.s2 = nn.MaxPool2d(pool_kernel_size, stride=2) # stride 2 and kernel 2x2 will half the size of output
    self.c3 = nn.Conv2d(6, 16, conv_kernel_size)
    self.s4 = self.s2 # i.e same as layer 2 max pooling
    # self.c5 = nn.Conv2d(16, 120, conv_kernel_size) # this doesn't seem to exist

    self.mlp = nn.Sequential(
        nn.Linear(400, 120),
        nn.Linear(120, 84),
        nn.Linear(84, 10)
    )

    self.softmax = nn.Softmax(dim=0)

  def forward(self, batch_img):
    y = self.c1(batch_img)
    y = self.s2(y)

    y = self.c3(y)
    y = self.s4(y)

    y = torch.flatten(y, start_dim=1) # starting from batch so don't flatten all...
    logits = self.mlp(y)
    y = self.softmax(y)

    return y


In [ ]:
class LeNet5_V2(nn.Module):

  def __init__(self, num_classes):
    super().__init__()

    self.c1 = nn.Conv2d(1, 6, conv_kernel_size) # with padding (0) and stride (1) default
    self.s2 = nn.AvgPool2d(pool_kernel_size, stride=2) # stride 2 and kernel 2x2 will half the size of output
    self.c3 = nn.Conv2d(6, 16, conv_kernel_size)
    self.s4 = self.s2 # i.e same as layer 2 max pooling
    # self.c5 = nn.Conv2d(16, 120, conv_kernel_size) # this doesn't seem to exist

    self.mlp = nn.Sequential(
        nn.Linear(400, 120),
        nn.ReLU(),
        nn.Linear(120, 84),
        nn.ReLU(),
        nn.Linear(84, 10)
    )

    self.ReLU = nn.ReLU()
    self.softmax = nn.Softmax(dim=0)

  def forward(self, batch_img):
    y = self.c1(batch_img)
    y = self.ReLU(y)
    y = self.s2(y)

    y = self.c3(y)
    y = self.ReLU(y)
    y = self.s4(y)

    y = torch.flatten(y, start_dim=1) # starting from batch so don't flatten all...
    logits = self.mlp(y)
    y = self.softmax(y)

    return y


In [ ]:
# model = LeNet5(num_classes).to(device)
model = LeNet5_V2(num_classes).to(device)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

total_step = len(train_loader)

In [ ]:
(images, labels) = train_loader.dataset[0]
images, labels

(tensor([[[-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          ...,
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242]]]),
 5)

In [ ]:
log_rate = 6000 / batch_size

def train(num_epochs):
  for epoch in range(num_epochs):
    for ind, (imgs, labels) in enumerate(train_loader):
      imgs = imgs.to(device)
      labels = labels.to(device)

      outputs = model(imgs)
      loss = loss_fn(outputs, labels)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      if(ind+1) % log_rate == 0:
        print('Epoch [{}/{}], Step [{}/{}, Loss: {:.4f}]'.format(epoch+1, num_epochs, ind+1, total_step, loss.item()))

In [ ]:
train(num_epochs)

In [ ]:
def test():
  model.eval()
  with torch.no_grad():
    correct = 0
    total = 0

    for imgs, labels in test_loader:
      imgs = imgs.to(device)
      labels = labels.to(device)

      outputs = model(imgs)
      # outputs.shape
      _, predicted = torch.max(outputs.data, 1)

      total += labels.size(0)
      correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total

    print(f'Accuracy of the network on the 10000 test images : {accuracy:.2f} %')

In [ ]:
len(test_loader)

40

In [ ]:
test()

Accuracy of the network on the 10000 test images : 10.03 %
